In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from scipy.sparse import csr_matrix

# Import data

In [4]:
datafile = './Protein_signature_rad_5.csv'

In [5]:
df = pd.read_csv(datafile, low_memory=False)

In [6]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,7063,7064,7065,7066,7067,7068,7069,7070,7071,7072
0,Q8L3C9@SwissProt,MLPKTM,MLPKTMK,MLPKTMKA,MLPKTMKAA,MLPKTMKAAV,MLPKTMKAAVI,LPKTMKAAVIR,PKTMKAAVIRE,KTMKAAVIREF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P39462@UniProt,MRAVRL,MRAVRLV,MRAVRLVE,MRAVRLVEI,MRAVRLVEIG,MRAVRLVEIGK,RAVRLVEIGKP,AVRLVEIGKPL,VRLVEIGKPLS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Q76HN6@UniProt,MEFDYL,MEFDYLI,MEFDYLIV,MEFDYLIVG,MEFDYLIVGA,MEFDYLIVGAG,EFDYLIVGAGS,FDYLIVGAGSA,DYLIVGAGSAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B2KJ46@UniProt,MGEIES,MGEIESY,MGEIESYC,MGEIESYCN,MGEIESYCNK,MGEIESYCNKE,GEIESYCNKEL,EIESYCNKELG,IESYCNKELGP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Q9P6C8@SwissProt,MPQFEI,MPQFEIP,MPQFEIPE,MPQFEIPEQ,MPQFEIPEQQ,MPQFEIPEQQW,PQFEIPEQQWA,QFEIPEQQWAQ,FEIPEQQWAQV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
labelfile = './training_6091_nsubs.csv'
df_label = pd.read_csv(labelfile, header=None)

In [8]:
y = df_label[2].values

# Generate a set of all signatures

In [9]:
df.iloc[0,:].values[1:]

array(['MLPKTM', 'MLPKTMK', 'MLPKTMKA', ..., nan, nan, nan], dtype=object)

In [10]:
main_sig_set = set()

def get_sigs(row):
    return set(row.dropna().values[1:])

for ind_set in df.apply(get_sigs, axis=1).values:
    main_sig_set.update(ind_set)

In [11]:
len(main_sig_set)

2718772

In [12]:
main_sig_dict = dict(zip(main_sig_set, range(len(main_sig_set))))

In [13]:
data = []
row = []
col = []
for idx,ind_set in enumerate(df.apply(get_sigs, axis=1).values):
    for sig in ind_set:
        col_val = main_sig_dict[sig]
#        X[idx, col_val] = 1
        data.append(1)
        row.append(idx)
        col.append(col_val)

In [14]:
X = csr_matrix((data, (row, col)), shape=(len(df), len(main_sig_dict)))

In [18]:
y_poisson = y

In [19]:
y_binary = y>1
y_binary = y_binary.astype(int)

In [20]:
np.unique(y_binary, return_counts=True)

(array([0, 1]), array([2280, 3811]))

In [21]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test, y_train_p, y_test_p = train_test_split(X, y_binary, y_poisson)

In [52]:
from sklearn.svm import SVC
clf = SVC(C=0.1, kernel='linear', gamma='auto')
clf.fit(X_train, y_train)

SVC(C=0.1, gamma='auto', kernel='linear')

In [53]:
clf.score(X_test, y_test)

0.6257386736703874

In [54]:
y_test_hat = clf.predict(X_test)

In [55]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [56]:
f1_score(y_test, y_test_hat)

0.7632890365448505

In [57]:
precision_score(y_test, y_test_hat, labels=[0], average='micro')

0.5230769230769231

In [58]:
recall_score(y_test, y_test_hat, labels=[0], average='micro')

0.059336823734729496

In [50]:
from sklearn.linear_model import PoissonRegressor

In [51]:
reg = PoissonRegressor()
reg.fit(X_train, y_train_p)

PoissonRegressor()

In [34]:
y_test_hat_p = reg.predict(X_test)

In [38]:
from sklearn.metrics import mean_squared_error, r2_score

In [37]:
mean_squared_error(y_test_p, y_test_hat_p)

78.96356519585113

In [39]:
r2_score(y_test_p, y_test_hat_p)

-0.004454109299340425